In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvidia-smi

Sat Jun 29 16:18:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.cuda.amp import GradScaler
import tiktoken

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANGPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        param_dict = {pn: p for pn, p in self.named_parameters() if p.requires_grad}
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T
        with open('/kaggle/input/inputt-txt/input.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2') 
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'loaded {len(self.tokens)} tokens')
        print(f'1 epoch = {len(self.tokens) // (B * T)} batches')
        self.current_position = 0
    
    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position: self.current_position + B * T + 1]
        x = (buf[:-1]).view(B, T)
        y = (buf[1:]).view(B, T)
        self.current_position += B*T
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

# Set up device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")

# Set seed for reproducibility
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

# Set up model
torch.set_float32_matmul_precision('high')
model = GPT(GPTConfig())
model.to(device)

# Set up training parameters
max_lr = 6e-4 
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 5000

def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

# Set up data loader
train_loader = DataLoaderLite(B=8, T=1024)

# Set up optimizer
optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=max_lr, device_type=device)

# Create a GradScaler for mixed precision training
scaler = GradScaler()

# ... [Previous code remains the same up to the training loop]

# Set up training parameters
max_lr = 6e-4 
min_lr = max_lr * 0.1
warmup_steps = 2000
max_steps = 1000000  # Increased to allow for more training steps
target_loss = 0.099999
patience = 1000  # Number of steps to wait for improvement before stopping

# ... [The rest of the setup code remains the same]

# Training loop
best_loss = float('inf')
steps_without_improvement = 0
step = 0

while True:
    t0 = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()

    # Runs the forward pass with autocasting
    with torch.autocast(device_type=device, dtype=torch.float16):
        logits, loss = model(x, y)

    # Scales loss. Calls backward() on scaled loss to create scaled gradients.
    scaler.scale(loss).backward()

    # Unscales gradients and calls or skips optimizer.step()
    scaler.unscale_(optimizer)
    
    # Clip gradients
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # If gradients are finite, call optimizer.step()
    # Otherwise, skip optimizer.step()
    scaler.step(optimizer)

    # Updates the scale for next iteration
    scaler.update()

    # Update learning rate
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
    torch.cuda.synchronize() 
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f'step {step} | loss: {loss.item():.6f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f} | norm: {norm:.2f} | lr: {lr:.6f}')

    # Check if we've reached the target loss
    if loss.item() < target_loss:
        print(f"Target loss achieved! Final loss: {loss.item():.6f}")
        break

    # Check for improvement
    if loss.item() < best_loss:
        best_loss = loss.item()
        steps_without_improvement = 0
    else:
        steps_without_improvement += 1

    # Stop if no improvement for 'patience' steps
    if steps_without_improvement >= patience:
        print(f"No improvement for {patience} steps. Stopping training. Best loss: {best_loss:.6f}")
        break

    # Stop if we've reached the maximum number of steps
    if step >= max_steps:
        print(f"Reached maximum steps. Stopping training. Best loss: {best_loss:.6f}")
        break

    step += 1

# Save the model
torch.save(model.state_dict(),'/kaggle/working/gpt_model.pth')
print("Training completed! Model saved as 'gpt_model.pth'")

using device: cuda
loaded 338025 tokens
1 epoch = 41 batches
num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True
step 0 | loss: 10.957500 | dt: 1724.61ms | tok/sec: 4750.05 | norm: 7.38 | lr: 0.000000
step 1 | loss: 9.402887 | dt: 1034.10ms | tok/sec: 7921.83 | norm: 4.01 | lr: 0.000001
step 2 | loss: 9.221437 | dt: 1035.96ms | tok/sec: 7907.67 | norm: 3.84 | lr: 0.000001
step 3 | loss: 9.275149 | dt: 1036.92ms | tok/sec: 7900.34 | norm: 3.84 | lr: 0.000001
step 4 | loss: 9.340334 | dt: 1038.28ms | tok/sec: 7889.94 | norm: 3.79 | lr: 0.000001
step 5 | loss: 9.450179 | dt: 1039.03ms | tok/sec: 7884.25 | norm: 3.77 | lr: 0.000002
step 6 | loss: 9.444682 | dt: 1034.06ms | tok/sec: 7922.19 | norm: 3.61 | lr: 0.000002
step 7 | loss: 9.384368 | dt: 1039.09ms | tok/sec: 7883.79 | norm: 3.45 | lr: 0.000002
step 8 | loss: 9.268996 | dt: 1039.44ms | tok/sec: 7881.18 | norm: 3.20 | lr: 0.000003
step 

In [ ]:
# Define the path where you want to save the model
model_path = '/kaggle/working//GPT_model.pth'

# Save the model state dictionary
torch.save(model.state_dict(), model_path)

print(f'Model saved successfully at {model_path}')
